## 당뇨병 여부 판단
- 이상치 처리 (Glucose, BloodPressure, SkinThickness, Insulin, BMI가 0인 값)

## [참고]작업형2 문구
- 출력을 원하실 경우 print() 함수 활용
- 예시) print(df.head())
- getcwd(), chdir() 등 작업 폴더 설정 불필요
- 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

### 데이터 파일 읽기 예제
- import pandas as pd
- X_test = pd.read_csv("data/X_test.csv")
- X_train = pd.read_csv("data/X_train.csv")
- y_train = pd.read_csv("data/y_train.csv")

### 사용자 코딩

### 답안 제출 참고
- 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
- pd.DataFrame({'cust_id': X_test.cust_id, 'gender': pred}).to_csv('003000000.csv', index=False)

In [134]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Outcome')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 9), (154, 9), (614, 2), (154, 2))

## Start

## 라이브러리 및 데이터 불러오기

In [135]:
# 라이브러리
import pandas as pd

In [136]:
# 데이터 불러오기 (생략)
X_train.shape, y_train.shape, X_test.shape

((614, 9), (614, 2), (154, 9))

## EDA

In [137]:
X_train.head()

,id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
147,147,2,106,64,35,119,30.5,1.400,34
344,344,8,95,72,0,0,36.8,0.485,57
390,390,1,100,66,29,196,32.0,0.444,42
150,150,1,136,74,50,204,37.4,0.399,24
132,132,3,170,64,37,225,34.5,0.356,30


In [138]:
y_train.head()

,id,Outcome
147,147,0
344,344,0
390,390,0
150,150,0
132,132,1


In [139]:
X_test.head()

,id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
258,258,1,193,50,16,375,25.9,0.655,24
220,220,0,177,60,29,478,34.6,1.072,21
438,438,1,97,70,15,0,18.2,0.147,21
130,130,4,173,70,14,168,29.7,0.361,33
730,730,3,130,78,23,79,28.4,0.323,34


In [140]:
# train 결측치 확인
X_train.isnull().sum()

id                          0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [141]:
# test 결측치 확인 
X_test.isnull().sum()

id                          0
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [142]:
# train 통계값 분석
X_train.describe()

,id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,386.978827,3.845277,120.705212,68.530945,20.982085,82.229642,31.948534,0.477471,33.319218
std,219.881344,3.388952,32.047977,19.761915,15.965031,116.525772,7.734385,0.339811,11.940953
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,196.500000,1.000000,99.000000,62.000000,0.000000,0.000000,27.400000,0.244250,24.000000
50%,388.000000,3.000000,117.000000,72.000000,23.000000,44.000000,32.250000,0.377000,29.000000
75%,575.750000,6.000000,140.000000,80.000000,33.000000,130.000000,36.600000,0.628500,41.000000
max,766.000000,17.000000,199.000000,114.000000,99.000000,846.000000,59.400000,2.420000,81.000000


## 데이터 전처리

In [143]:
#이상치 처리 (Glucose, BloodPressure, SkinThickness, Insulin, BMI가 0인 값)
#train
print("Glucose:", len(X_train[X_train['Glucose'] == 0]))
print("BloodPressure:", len(X_train[X_train['BloodPressure'] == 0]))
print("SkinThickness:", len(X_train[X_train['SkinThickness'] == 0]))
print("Insulin:", len(X_train[X_train['Insulin'] == 0]))
print("BMI:", len(X_train[X_train['BMI'] == 0]))

Glucose: 5
BloodPressure: 31
SkinThickness: 175
Insulin: 287
BMI: 9


In [144]:
#Test
print("Glucose:", len(X_test[X_test['Glucose'] == 0]))
print("BloodPressure:", len(X_test[X_test['BloodPressure'] == 0]))
print("SkinThickness:", len(X_test[X_test['SkinThickness'] == 0]))
print("Insulin:", len(X_test[X_test['Insulin'] == 0]))
print("BMI:", len(X_test[X_test['BMI'] == 0]))

Glucose: 0
BloodPressure: 4
SkinThickness: 52
Insulin: 87
BMI: 2


트레인에만 0이 있는 포도당(Glucose)는 삭제
나머지는 평균값으로 대체 하여 이상치를 처리함

In [145]:
# 포도당 이상치 삭제
del_idx = (X_train[X_train['Glucose'] == 0]).index
del_idx

Int64Index([182, 75, 342, 502, 349], dtype='int64')

In [146]:
print('Glucose 이상치 삭제 전: ', X_train.shape, y_train.shape)
X_train = X_train.drop(index = del_idx, axis = 0)
y_train = y_train.drop(index = del_idx, axis = 0)
print("Glucose 이상치 삭제 후: ", X_train.shape, y_train.shape)

Glucose 이상치 삭제 전:  (614, 9) (614, 2)
Glucose 이상치 삭제 후:  (609, 9) (609, 2)


In [147]:
# 포도당을 제외한 이상치, 평균값으로 대체
cols = ['BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
cols_mean = X_train[cols].mean()
X_train[cols] = X_train[cols].replace(0, cols_mean)


cols_mean2 = X_test[cols].mean()
X_test[cols] = X_test[cols].replace(0, cols_mean2)
X_test.head()

,id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
258,258,1,193,50.0,16.0,375.00000,25.9,0.655,24
220,220,0,177,60.0,29.0,478.00000,34.6,1.072,21
438,438,1,97,70.0,15.0,70.11039,18.2,0.147,21
130,130,4,173,70.0,14.0,168.00000,29.7,0.361,33
730,730,3,130,78.0,23.0,79.00000,28.4,0.323,34


In [149]:
# id열 제거
X_train = X_train.drop(['id'], axis=1)
X_test = X_test.drop(['id'], axis=1)
print(X_train.columns)
print(X_test.columns)

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')
Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')


In [150]:
# 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X_train)
test = scaler.transform(X_test)


In [151]:
from sklearn.svm import SVC
model = SVC(gamma = 0.01, random_state=2022)
model.fit(X, y_train['Outcome'])
predictions = model.predict(test)

In [152]:
# 오버피팅 되었을 경우에 점수가 잘나올 수 있음 (객관적인 평가 아님, 밸리데이션 데이터로 평가 필요함)
round(model.score(X, y_train['Outcome']) * 100, 2) 

78.0

In [153]:
output = pd.DataFrame({'idx': X_test.index, 'Outcome': predictions})
output.head()

,idx,Outcome
0,258,1
1,220,1
2,438,0
3,130,1
4,730,0


In [154]:
# 수험번호.csv로 출력
output.to_csv('1234567.csv', index=False)

## 결과 체점 (수험자는 알 수 없는 부분임)

In [155]:
round(model.score(test, y_test['Outcome']) * 100, 2)

76.62